# Getting the data

https://scikit-learn.org/stable/modules/model_evaluation.html

In [1]:
!git clone https://github.com/YasaminAbbaszadegan/NLP_Forum_Recomendation_Engine.git

Cloning into 'NLP_Forum_Recomendation_Engine'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 61 (delta 29), reused 43 (delta 11), pack-reused 0
Unpacking objects: 100% (61/61), done.


In [2]:
%cd /content/NLP_Forum_Recomendation_Engine/Dataset

/content/NLP_Forum_Recomendation_Engine/Dataset


In [3]:
ls

tudiabetes_20.csv


In [4]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [260]:

df=pd.read_csv('tudiabetes_20.csv',index_col=0) #MyDF_final_main_comment.csv

In [22]:
#df=df.drop(['tags'],axis=1)
df

,Topic Title,Category,Tags,Leading Post,Post Replies,Created_at,Likes,Views,Replies
0,About the Social Clubs category,Social Clubs,[],Social Clubs are places for people with simila...,[],"Jul 26, 2018 9:24 pm",0,722,2
1,Latest Movie You’ve Seen?,Social Clubs,['Movies and Entertainment Media'],"Hi Everyone,\nWhat is the latest movie that yo...",['My wife and I see about one movie a week. L...,"Mar 12, 2019 12:45 am",0,578,5
2,Anyone doing the Broad St. Run in philly,Social Clubs,['Running'],got my notification yesterday. time to get bac...,[],0,0,0,0
3,Embroidery…. just starting out,Social Clubs,['Carefully Crafted'],Love love love creating!\nAfter years of not s...,"['That’s beautiful!', 'Thanks, so loving a dar...","Feb 4, 2019 11:31 am",4,505,2
4,Home theatre,Social Clubs,['Movies and Entertainment Media'],If anyone is approaching retirement and wonder...,[],0,0,0,0
...,...,...,...,...,...,...,...,...,...
3694,Frequent Occlusions with Tandem X2,Diabetes Technology,[],A few months ago my son switched from a Medtro...,"['What infusion sets are he using?', 'The Tand...","Dec 11, 2019 3:52 pm",7,700,15
3695,Any loopers using this button?,Diabetes Technology,['DIY Closed Loop Systems'],Anybody using this button (right of the meal b...,"['I use this button many times, every day. It’...","Dec 7, 2019 11:46 am",0,645,24
3696,Which infusion set?,Diabetes Technology,['Insulin Pumps'],I have Tandem T2 pump. Started using the steel...,['I use the Autosoft 30. A spring sends it in...,"Dec 12, 2018 6:10 pm",7,1.8k,17
3697,Availability of diabetes products in Canada. I...,Diabetes Technology,[],So I love new advancements in D-technology. Bu...,['Yes - frustrated.\nI would like to try Tresi...,"Oct 6, 2016 9:14 pm",16,7.0k,33


# Remove '[ ]' in Tags and Post Replies

In [261]:
df['Tags'] = df['Tags'].str.replace("[", "")
df['Tags'] = df['Tags'].str.replace("]", "")
df['Tags'] = df['Tags'].str.replace("'", "")
df['Post Replies'] = df['Post Replies'].str.replace("[", "")
df['Post Replies'] = df['Post Replies'].str.replace("]", "")
df['Post Replies'] = df['Post Replies'].str.replace("'", "")


In [85]:
df.head()

,Topic Title,Category,Tags,Leading Post,Post Replies,Created_at,Likes,Views,Replies
0,About the Social Clubs category,Social Clubs,,Social Clubs are places for people with simila...,,"Jul 26, 2018 9:24 pm",0,722,2
1,Latest Movie You’ve Seen?,Social Clubs,Movies and Entertainment Media,"Hi Everyone,\nWhat is the latest movie that yo...",My wife and I see about one movie a week. Las...,"Mar 12, 2019 12:45 am",0,578,5
2,Anyone doing the Broad St. Run in philly,Social Clubs,Running,got my notification yesterday. time to get bac...,,0,0,0,0
3,Embroidery…. just starting out,Social Clubs,Carefully Crafted,Love love love creating!\nAfter years of not s...,"That’s beautiful!, Thanks, so loving a dark ba...","Feb 4, 2019 11:31 am",4,505,2
4,Home theatre,Social Clubs,Movies and Entertainment Media,If anyone is approaching retirement and wonder...,,0,0,0,0


In [25]:
df['Category'].unique()

array(['Social Clubs', 'TuDiabetes Website', 'Welcome', 'Type 1 and LADA',
       'Type 2', 'Diabetes and Pregnancy', 'Community', 'Food',
       'Treatment', 'Diabetes Technology'], dtype=object)

In [10]:
df['Category'].nunique()

10

# Remove df['category']='Social Clubs','Welcome','Tudiabetes website'

In [300]:
df_cat = df[df.Category != 'Social Clubs']
df_cat = df_cat[df.Category != 'TuDiabetes Website']
df_cat = df_cat[df.Category != 'Welcome']



In [40]:
print("After removing 'Social Clubs', 'TuDiabetes Website', 'Welcome' categories we remove vvv rows ")
len(df)-len(df_cat)

After removing 'Social Clubs', 'TuDiabetes Website', 'Welcome' categories we remove vvv rows 


192

In [263]:
df=df_cat.drop(labels=['Created_at','Likes','Views','Replies'],axis=1)

In [265]:
df['comments']=df['Leading Post']+df['Post Replies']

In [44]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=bd14c85011c1c72b7643a8c78bf02eff9eaf6653befd30391c5d2f73d5564189
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [45]:
from rake_nltk import Rake
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [266]:
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
from bs4 import BeautifulSoup
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\'\“\”\’\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
DIGITS_RE=re.compile('\w*\d\w*')


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """

    text = str(text).lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = text.replace(r'\n', ' ')
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = DIGITS_RE.sub('', text)
    return text

In [267]:
df['comments'] = df['comments'].apply(clean_text)

In [268]:
df['Tags']=df['Tags'].apply(clean_text)

In [269]:
df['Category']=df['Category'].apply(clean_text)

In [270]:
df.head()

,Topic Title,Category,Tags,Leading Post,Post Replies,comments
192,Type 1 Diabetes Resources,type and lada,,Information on this page represents a sample o...,,information on this page represents a sample o...
193,About the Type 1 and LADA category,type and lada,,The Type 1 Diabetes and LADA category is inten...,,the type diabetes and lada category is intend...
194,Are you concerned about Covid-19 Vaccine might...,type and lada,,I am wondering if others with multiple autoimm...,"Yeah. However, given that this is a pandemic, ...",i am wondering if others with multiple autoimm...
195,Covid Vaccine - are you getting it?,type and lada,,Hello!\nI wanted to get people who have type 1...,I am Type 2 but the question is just a relevan...,helloi wanted to get people who have type the...
196,Bolusing for pasta - big fail,type and lada,,How do you guys bolus for pasta? I had some e...,"With pasta (any kind of spaghetti, light on th...",how do you guys bolus for pasta i had some el...


In [271]:
df['Key_words'] = ''
r = Rake()
for index, row in df.iterrows():
    r.extract_keywords_from_text(row['comments'])
    key_words_dict_scores = r.get_word_degrees() 

    """"
   ##get_word_degrees() 
    Method to fetch the degree of words in the given text. Degree can be
    defined as sum of co-occurances of the word with other words in the
    given text."""
    row['Key_words'] = list(key_words_dict_scores.keys())

In [272]:
df.head()

,Topic Title,Category,Tags,Leading Post,Post Replies,comments,Key_words
192,Type 1 Diabetes Resources,type and lada,,Information on this page represents a sample o...,,information on this page represents a sample o...,"[begin, expertise, related, day, highly, infor..."
193,About the Type 1 and LADA category,type and lada,,The Type 1 Diabetes and LADA category is inten...,,the type diabetes and lada category is intend...,"[wish, one, discussion, specifically, write, p..."
194,Are you concerned about Covid-19 Vaccine might...,type and lada,,I am wondering if others with multiple autoimm...,"Yeah. However, given that this is a pandemic, ...",i am wondering if others with multiple autoimm...,"[even, much, lot, would, also, agree, covid, g..."
195,Covid Vaccine - are you getting it?,type and lada,,Hello!\nI wanted to get people who have type 1...,I am Type 2 but the question is just a relevan...,helloi wanted to get people who have type the...,"[ability, illnesses, toothache, dentist, may, ..."
196,Bolusing for pasta - big fail,type and lada,,How do you guys bolus for pasta? I had some e...,"With pasta (any kind of spaghetti, light on th...",how do you guys bolus for pasta i had some el...,"[eat, g, lot, bother, measuring, confidence, a..."


In [273]:
df_test=df

In [274]:
# to combine 3 lists (3 columns) of key words into 1 sentence under Bag_of_words column



df_test['Bag_of_words'] = ''
columns = ['Category', 'Tags']

for index, row in df_test.iterrows():
    words = ''
    for col in columns:
        words += ''.join(row[col]) + ''
    row['Bag_of_words'] = words
    
# strip white spaces infront and behind, replace multiple whitespaces (if any)
df_test['Bag_of_words'] = df_test['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')
# df_test['Bag_of_words']=df_test['Bag_of_words']+str(df['Key_words'])
df_test = df_test[['Topic Title','Bag_of_words']]
df_test.head()

,Topic Title,Bag_of_words
192,Type 1 Diabetes Resources,type and lada
193,About the Type 1 and LADA category,type and lada
194,Are you concerned about Covid-19 Vaccine might...,type and lada
195,Covid Vaccine - are you getting it?,type and lada
196,Bolusing for pasta - big fail,type and lada


In [275]:
df_test1=df

In [276]:
df_test1['Bag_of_words'] = ''
columns = ['Key_words']

for index, row in df_test1.iterrows():
    words = ''
    words += ' '.join(row['Key_words']) + ' '
    row['Bag_of_words'] = words
    
# strip white spaces infront and behind, replace multiple whitespaces (if any)
df_test1['Bag_of_words'] = df_test1['Bag_of_words'].str.strip().str.replace('   ', ' ').str.replace('  ', ' ')
# df_test['Bag_of_words']=df_test['Bag_of_words']+str(df['Key_words'])
df_test1 = df_test1[['Topic Title','Bag_of_words']]
df_test1.head()

,Topic Title,Bag_of_words
192,Type 1 Diabetes Resources,begin expertise related day highly informative...
193,About the Type 1 and LADA category,wish one discussion specifically write people ...
194,Are you concerned about Covid-19 Vaccine might...,even much lot would also agree covid getting a...
195,Covid Vaccine - are you getting it?,ability illnesses toothache dentist may get pf...
196,Bolusing for pasta - big fail,eat g lot bother measuring confidence ability ...


In [277]:
df_test['Bag_of_words']=df_test['Bag_of_words']+' '+df_test1['Bag_of_words']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [278]:
df_test.head()

,Topic Title,Bag_of_words
192,Type 1 Diabetes Resources,type and lada begin expertise related day high...
193,About the Type 1 and LADA category,type and lada wish one discussion specifically...
194,Are you concerned about Covid-19 Vaccine might...,type and lada even much lot would also agree c...
195,Covid Vaccine - are you getting it?,type and lada ability illnesses toothache dent...
196,Bolusing for pasta - big fail,type and lada eat g lot bother measuring confi...


## Recomendation Model

In [279]:
count = CountVectorizer()
count_matrix = count.fit_transform(df_test['Bag_of_words'])
count_matrix


<3507x60259 sparse matrix of type '<class 'numpy.int64'>'
	with 979925 stored elements in Compressed Sparse Row format>

In [280]:
 #to generate the cosine similarity matrix (size 5464 x 5464)
# rows represent all topic-titles; columns represent all topic-titles
# cosine similarity: similarity = cos(angle) = range from 0 (different) to 1 (similar)
# all the numbers on the diagonal are 1 because every topic-title is identical to itself (cosine value is 1 means exactly identical)
# matrix is also symmetrical because the similarity between A and B is the same as the similarity between B and A.
# for other values eg 0.1578947, movie x and movie y has similarity value of 0.1578947

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)


[[1.         0.22237479 0.15022039 ... 0.05555899 0.06670904 0.10219734]
 [0.22237479 1.         0.13135266 ... 0.01873829 0.06428243 0.07385987]
 [0.15022039 0.13135266 1.         ... 0.16033771 0.18334824 0.20943331]
 ...
 [0.05555899 0.01873829 0.16033771 ... 1.         0.18549963 0.21405987]
 [0.06670904 0.06428243 0.18334824 ... 0.18549963 1.         0.23211917]
 [0.10219734 0.07385987 0.20943331 ... 0.21405987 0.23211917 1.        ]]


In [299]:
# to create a Series for topic-titles which can be used as indices (each index is mapped to a topic-title)
indices = pd.Series(df_test['Topic Title'])


In [372]:

# this function takes in a topic-title as input and returns the top 10 recommended (similar) topic-titles

def recommend(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_movies.append(list(df_test['Topic Title'])[i])
        
    return recommended_movies

recommend('Type 1 Diabetes Resources')

['What did you eat today?',
 'High fasting sugars',
 'T2D, a1c 11, scared out of my mind',
 'Type 2 issues',
 'LCHF help!',
 'Any good breakfast ideas?',
 'Tips for getting started on a LC diet',
 'Advice/encouragement needed',
 'Food ideas for sensitive stomach',
 'Diet Questions']

In [321]:
df_test['Topic Title'][1:2]

193    About the Type 1 and LADA category
Name: Topic Title, dtype: object

In [255]:
recommend('New Book For Kids With Type 1')

['Will we ever have treatments without numerous devices beeping at us?',
 'Xdrip vs. Dexcom app? (G6)',
 'Suggestions to make a Dexcom better',
 'Glimpse the future',
 '670g Experience?',
 'Life of a G6 transmitter',
 'Thinking about Omnipods',
 'Tandem connect',
 'Nightly raising, raising hello',
 'Starting on G6 and Basal IQ']

In [284]:
recommend('T2D, a1c 11, scared out of my mind')

['Cholesterol and Statins causing joint pains - need advice',
 'Statins',
 'New Hashimoto’s Dx',
 'Have any of you had a heart attack?',
 'Possible LADA and yo-yo numbers',
 'Anyone negative for GAD but positive for other antibodies?',
 'Cholesterol Report Tool',
 'Statin Intolerance - It is Real but we have a CURE!',
 'Type 1 father always worried',
 'Pre-Diabetes at 19 why?']

In [285]:
recommend('Have any of you had a heart attack?')

['Newly pregnant, sick with bronchitis, panicking about highs: am I over-reacting?',
 'Early Pregnancy BG Spikes?',
 '8 weeks pregnant and having lots of lows',
 'Aggressive corrections?',
 'Questioning my health/Potential Pre-Diabetes?',
 'Getting Prepared',
 'Fasting blood sugar during pregnancy',
 'Miscarriage, what went wrong?',
 'First Pregnancy with Type 1 Diabetes',
 'Silly questions']

In [286]:
recommend('Newly pregnant, sick with bronchitis, panicking about highs: am I over-reacting?')

['Who helped manage your blood sugars during pregnancy?',
 'What Q’s should you ask at the first visit with the High Risk OB (Maternal-Fetal Medicine)?',
 'Need Pregnancy Tips- Please help!',
 'Unexpected pregnancy',
 'New diabetic and pregnancy',
 'I’m pregnant… on accident',
 'a1c before/during pregnancy?',
 'Do I need a high risk OBGYN prior to conceiving?',
 'Advice for 2nd baby',
 'Medical advice about becoming pregnant']

In [287]:
recommend('Medical advice about becoming pregnant')

['Very HIGH blood sugars in Early pregnancy instead of LOWS?',
 'Need advice for pre-conception blood sugar targets',
 'Is it really possible to maintain such tight control?!',
 '4 weeks pregnant, A1c mid-7’s - freaking out!',
 '100% increase of basal rate at 4 weeks?!',
 'So nervous',
 'Getting ready for the next step!',
 'Second Baby- Out of control Diabetic',
 'Pregnancy while honeymooning',
 'I have officially entered the insulin resistance nightmare']

In [292]:
recommend('Frozen Banana Coconut Cups')

['Sugar free tablet',
 'Sweeteners',
 'Fructose Metabolism and Insulin',
 'Glycemic index food labeling?',
 'Barley',
 'Sugar free cake?',
 'Bolusing for pasta - big fail',
 'Does peanut butter raise your blood sugar?',
 'Not all carbohydrates are created equal…?',
 'Type 1’s, how many carbs do you eat/day and what is your A1c?']

In [294]:
recommend('Carb-addict patch, like smokers’ patch')

['Healthy chocolate mousse?!?!',
 'Dried Plum and Pistachio Chia Pudding',
 'Has anyone tried Oopsies as an alternative to bread?',
 'Pear and Brazil Nut Chocolate Brownies',
 'Desserts',
 'What’s Your Personal Fave Low Carb?',
 'Cheddar Ranch Roasted Cauliflower',
 'Fast chia pudding recipe please?',
 'Low Carb German Chocolate Brownies',
 'November is Recipe Share Month!']

In [295]:
recommend('Keto Diet?')

['Community Roll Call! Your best under 25 grams of carbohydrate meal ideas! Post em’!',
 'Terry on Juicebox Podcast again!',
 'New to the World of Diabetes',
 'Insulin Resistance Getting Crazy?',
 'G6 Sensor Code',
 'Using a Planner to keep track of diabetes',
 'Anyone familiar with Humalog 200',
 'Nuco coconut wraps',
 'About Diabetic Diet Schedule',
 'Diagnosed with type 2 at age 12, but I call myself a diabetic hybrid']

In [296]:
recommend('Bread recipe?')

['Peanut butter “bad?”',
 'Help I don’t know what’s going on',
 'Is a low-carb diet a must?',
 'Carb question regarding potatoes',
 'Extra use of fast-acting insulin',
 'Breakfast ideas',
 'BS going up 2 hours after waking',
 'I should be testing how many times?',
 'What do you consider a low carb daily diet?',
 'Low glycemic fruits suggestions']

In [298]:
recommend('Is Dexcom app working with IOS 14 on iPhone?')

['CGM Site Scars',
 'What’s going on?',
 'Any suggestions on why I keep hitting capillaries lately?',
 'Trying out needle infusion sets',
 'Project hyperlipotrophy prevention!',
 'Makes No Sense',
 'Site problems',
 'It’s Happening again!',
 'Longer lasting infusion sets?',
 'Changing Infusion set hitting veins']

In [356]:
# def recommend(title,n_top, cosine_sim = cosine_sim):
#     recommended_movies = []
#     idx = indices[indices == title].index[0]
#     score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
#     top_10_indices = list(score_series.iloc[0:n_top].index)
    
#     for i in top_10_indices:
#         recommended_movies.append(list(df_test['Topic Title'])[i])
        
#     return recommended_movies

# recommend('Type 1 Diabetes Resources',3)

['What did you eat yesterday?',
 'What did you eat today?',
 'High fasting sugars']